In [25]:
import builtins
import geopandas as gpd
import getpass
import hvplot.xarray
from IPython.display import Image
import matplotlib.patches
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import netCDF4
import netrc
import numpy as np
import os
import osgeo.gdal, osgeo.osr
import pyproj
import rasterio
import rasterio.features
import rasterio.warp
import rioxarray
import re
import warnings
import xarray as xr
import h5py

import hvplot.pandas # Enable 'hvplot' accessor on Geodataframes for interactive plotting
import h5py
import pandas as pd

# import routines for this notebook
# import utilities
# turn off warnings
warnings.filterwarnings('ignore')

In [70]:
# Loading data
filename = "/tmp/test/processed_ATL07-02_20190927130752_00100501_005_01.h5"
ds1 = xr.open_dataset(filename,group='gt2r/sea_ice_segments/heights')  
ds = xr.open_dataset(filename,group='gt2r/sea_ice_segments')  

In [75]:
# For simplicity take first 1000 points into a Geopandas Geodataframe
# df1 = ds1.isel(delta_time=slice(0,221000)).to_dataframe()
# df = ds.isel(delta_time=slice(0,221000)).to_dataframe()
df1 = ds1.to_dataframe()
df = ds.to_dataframe()
# NOTE: that the Coordinate Reference System is not propagated by xarray from the HDF5 metadata, so we have to assign it again!
gf = gpd.GeoDataFrame(df1, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs='epsg:7661')
gf.head()

,across_track_distance,height_segment_asr_calc,height_segment_confidence,height_segment_fit_quality_flag,height_segment_height,height_segment_htcorr_skew,height_segment_length_seg,height_segment_n_pulse_seg,height_segment_n_pulse_seg_used,height_segment_quality,height_segment_rms,height_segment_ssh_flag,height_segment_surface_error_est,height_segment_type,height_segment_w_gaussian,geometry
delta_time,,,,,,,,,,,,,,,,
0,-46.033504,NaN,NaN,NaN,NaN,NaN,714.825256,12201,12201,0,NaN,0,NaN,1,NaN,POINT (132.54060 -66.16790)
1,-46.033504,NaN,NaN,NaN,NaN,NaN,714.825256,6101,6101,0,NaN,0,NaN,1,NaN,POINT (132.54060 -66.16790)
2,-46.033504,NaN,NaN,NaN,NaN,NaN,714.825256,3051,3051,0,NaN,0,NaN,1,NaN,POINT (132.54060 -66.16790)
3,-46.033504,NaN,NaN,NaN,NaN,NaN,714.825256,1526,1526,0,NaN,0,NaN,1,NaN,POINT (132.54060 -66.16790)
4,-45.999428,NaN,NaN,NaN,NaN,NaN,557.730225,791,791,0,NaN,0,NaN,1,NaN,POINT (132.53967 -66.16458)


In [76]:

# Basic plot
# gf.plot.scatter(x='segment_id',y='h_li');
#gf.plot.scatter(x='longitude', y='latitude', c='h_li');

gf.hvplot.points(c='height_segment_height', coastline=True, tiles=True, cmap='viridis', data_aspect=0.1)

:Overlay
   .Tiles.I     :Tiles   [x,y]
   .Points.I    :Points   [Longitude,Latitude]   (height_segment_height)
   .Coastline.I :Feature   [Longitude,Latitude]

In [43]:
# Creating pandas array (optional)

# Load the ICESat-2 data. We will just look at the central beams (GT2R/L)
is2_file = '/tmp/test/processed_ATL07-02_20190927130752_00100501_005_01.h5'
with h5py.File(is2_file, 'r') as f:
    is2_gt2r = pd.DataFrame(data={'lat': f['gt2r/sea_ice_segments/latitude'][:],
                                  'lon': f['gt2r/sea_ice_segments/longitude'][:],
                                  'ice_conc': f['gt2r/sea_ice_segments/stats/ice_conc'][:]})
    # is2_gt2l = pd.DataFrame(data={'lat': f['gt2l/land_ice_segments/latitude'][:],
    #                               'lon': f['gt2l/land_ice_segments/longitude'][:],
    #                               'elev': f['gt2l/land_ice_segments/ice_conc'][:]})
    